### Summary
For my final project I have chosen to compete in a kaggle competition. <br>
The competition is being put on by HM clothing and the goal is to predict what shoppers will buy in the next 7 days after their last purchase.

The competition details are here -> https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations


### Data

The data